<a href="https://colab.research.google.com/github/JonJonesBR/MESTRE_RPG_GEMINI/blob/main/RPG_AUDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai edge-tts playsound nest_asyncio gradio Pillow

import gradio as gr
import google.generativeai as genai
import time
import os
import io
import sys
import asyncio
import re
import base64 # Importado para codificar o áudio

# Configuração para permitir loops asyncio aninhados
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("nest_asyncio aplicado.")
except ImportError:
    print("AVISO: nest_asyncio não encontrado.")
except RuntimeError as e:
    if "cannot apply nest_asyncio" in str(e) or "loop is already running" in str(e):
        print(f"nest_asyncio: {e}. Provavelmente já está gerenciado pelo ambiente (ex: Gradio).")
    else:
        print(f"AVISO: nest_asyncio: {e}")

_edge_tts_available = False
try:
    import edge_tts
    _edge_tts_available = True
except ImportError:
    print("AVISO: Biblioteca edge-tts não encontrada. A funcionalidade de voz não estará disponível.")

API_KEY_FILENAME = ".gemini_api_key.txt"

DURATION_OPTIONS_CONFIG = {
    "1": {"name": "Curta (objetivo direto)", "id": "curta", "turns": 10},
    "2": {"name": "Média (mais exploração)", "id": "media", "turns": 20},
    "3": {"name": "Longa (desenvolvimento e clímax)", "id": "longa", "turns": 40}
}

model_text = None
MODEL_NAME_TEXT = "gemini-2.0-flash-lite"

generation_config_text = {"temperature": 0.8, "top_p": 0.95, "top_k": 40}
safety_settings_text = [
    {"category": c, "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
    for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]
]

# --- Música de Fundo do Arquivo Local do Colab ---
MUSIC_FILE_PATH = "/content/epic_rpg_music.mp3" # Caminho no Colab
music_data_url = None
background_music_html = ""

if os.path.exists(MUSIC_FILE_PATH):
    try:
        with open(MUSIC_FILE_PATH, "rb") as f_audio:
            audio_bytes = f_audio.read()
        music_base64 = base64.b64encode(audio_bytes).decode('utf-8')
        music_data_url = f"data:audio/mp3;base64,{music_base64}"
        file_size_mb = len(audio_bytes) / (1024 * 1024)
        print(f"Arquivo de música '{MUSIC_FILE_PATH}' ({file_size_mb:.2f} MB) carregado e codificado em Base64.")

        background_music_html = f"""
        <audio id="backgroundMusic" src="{music_data_url}" loop>
          Seu navegador não suporta o elemento de áudio.
        </audio>
        <script>
          var music = document.getElementById('backgroundMusic');
          if (music) {{
            music.volume = 0.25; // Volume em 25%. Ajuste entre 0.0 (mudo) e 1.0 (máximo). 0.5 para 50%.

            function tryPlayMusic() {{
                if (music.paused) {{
                    music.play().then(() => {{
                        console.log("Música de fundo iniciada.");
                        // Remove os listeners após sucesso para não tentar repetidamente sem necessidade
                        document.body.removeEventListener('click', tryPlayMusic);
                        document.body.removeEventListener('touchstart', tryPlayMusic);
                    }}).catch(e => {{
                        console.warn("Tentativa de play da música falhou (normal se autoplay bloqueado):", e.message);
                        // Mantém os listeners se o play falhou, para tentar na próxima interação
                    }});
                }}
            }}

            // Tenta tocar quando os metadados da música estão carregados ou após uma interação
            if (music.readyState >= 2) {{ // HTMLMediaElement.HAVE_CURRENT_DATA
                 tryPlayMusic();
            }} else {{
                 music.addEventListener('canplaythrough', tryPlayMusic, {{ once: true }});
            }}
            // Adiciona listeners para a primeira interação do usuário como fallback robusto
            document.body.addEventListener('click', tryPlayMusic, {{ once: true }});
            document.body.addEventListener('touchstart', tryPlayMusic, {{ once: true }});

            // Uma tentativa de autoplay direto, que pode ser bloqueada
            // music.autoplay = true; // A tag 'loop' já está presente
            // A tentativa de play é feita pelo script acima agora.

          }} else {{
            console.error("Elemento de áudio 'backgroundMusic' não encontrado no DOM.");
          }}
        </script>
        """
    except Exception as e:
        print(f"Erro ao carregar ou codificar o arquivo de música local '{MUSIC_FILE_PATH}': {e}")
        background_music_html = f""
else:
    print(f"AVISO: Arquivo de música '{MUSIC_FILE_PATH}' não encontrado. A música de fundo não será reproduzida.")
    # O valor padrão de background_music_html já está definido


async def _speak_text_async_gradio(text, voice_name="pt-BR-ThalitaNeural", rate="+0%", pitch="+0Hz"):
    if not _edge_tts_available or not text or not text.strip(): return None
    text_cleaned = re.sub(r'[\*#_`]', ' ', text)
    text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    if not text_cleaned: return None
    try:
        communicate = edge_tts.Communicate(text_cleaned, voice_name, rate=rate, pitch=pitch)
        temp_filename = f"temp_audio_{int(time.time()*1000)}.mp3"
        audio_bytes_io = io.BytesIO()
        async for chunk in communicate.stream():
            if chunk["type"] == "audio": audio_bytes_io.write(chunk["data"])
        audio_bytes_io.seek(0)
        if audio_bytes_io.getbuffer().nbytes > 0:
            with open(temp_filename, "wb") as f: f.write(audio_bytes_io.getvalue())
            return temp_filename
        return None
    except Exception as e_tts:
        print(f"[TTS Edge Erro Geral: {e_tts}]")
        return None

async def speak_text_for_gradio(text, voice_name="pt-BR-ThalitaNeural"):
    if not _edge_tts_available or not text or not text.strip(): return None
    return await _speak_text_async_gradio(text, voice_name)

def configure_gemini_api_gradio_direct(api_key_input):
    global model_text
    feedback_list = []
    api_key_val = api_key_input.strip() if api_key_input else ""

    if not api_key_val or not api_key_val.startswith("AIza") or len(api_key_val) <= 20:
        feedback_list.append("Formato de chave API inválido.")
        if os.path.exists(API_KEY_FILENAME):
            try: os.remove(API_KEY_FILENAME); feedback_list.append(f"Arquivo '{API_KEY_FILENAME}' removido.")
            except Exception as e_rem: feedback_list.append(f"Erro ao remover '{API_KEY_FILENAME}': {e_rem}")
        return "\n".join(feedback_list), gr.update(visible=True), gr.update(visible=False)

    try:
        with open(API_KEY_FILENAME, "w") as f: f.write(api_key_val)
        feedback_list.append(f"Chave API salva em '{API_KEY_FILENAME}'.")
    except Exception as e:
        feedback_list.append(f"Não foi possível salvar a chave API: {e}.")

    try:
        genai.configure(api_key=api_key_val)
        model_text = genai.GenerativeModel(
            model_name=MODEL_NAME_TEXT,
            generation_config=generation_config_text,
            safety_settings=safety_settings_text
        )
        feedback_list.append(f"API Gemini configurada. Modelo de Texto '{model_text.model_name}' pronto.")
        return "\n".join(feedback_list), gr.update(visible=False), gr.update(visible=True)
    except Exception as e:
        feedback_list.append(f"Erro CRÍTICO ao configurar API Gemini (Modelo de Texto: {MODEL_NAME_TEXT}): {e}")
        if os.path.exists(API_KEY_FILENAME):
            try: os.remove(API_KEY_FILENAME); feedback_list.append(f"Arquivo '{API_KEY_FILENAME}' removido devido a erro crítico.")
            except Exception as e_rem: feedback_list.append(f"Erro ao remover chave: {e_rem}")
        model_text = None
        return "\n".join(feedback_list), gr.update(visible=True), gr.update(visible=False)


def load_api_key_on_startup_gradio():
    global model_text
    api_key_loaded_val = ""
    feedback = []
    api_section_vis = gr.update(visible=True)
    game_setup_vis = gr.update(visible=False)

    if os.path.exists(API_KEY_FILENAME):
        try:
            with open(API_KEY_FILENAME, "r") as f: api_key_loaded_val = f.read().strip()
            if api_key_loaded_val and api_key_loaded_val.startswith("AIza") and len(api_key_loaded_val) > 20:
                feedback.append("Chave API carregada de arquivo local.")
                config_feedback_str, api_section_vis, game_setup_vis = configure_gemini_api_gradio_direct(api_key_loaded_val)
                feedback.append(config_feedback_str)
            else:
                if os.path.exists(API_KEY_FILENAME): os.remove(API_KEY_FILENAME)
                feedback.append(f"Chave em '{API_KEY_FILENAME}' inválida, arquivo removido.")
                api_key_loaded_val = ""
        except Exception as e:
            feedback.append(f"Erro ao ler '{API_KEY_FILENAME}': {e}")
            api_key_loaded_val = ""

    if not model_text:
        if not any("API Gemini configurada" in s for s in feedback) and not any("Erro CRÍTICO" in s for s in feedback):
             feedback.append("Insira sua chave API Gemini.")
        api_section_vis = gr.update(visible=True)
        game_setup_vis = gr.update(visible=False)

    return "\n".join(feedback), api_section_vis, game_setup_vis, api_key_loaded_val

def mestre_responde(prompt_para_o_mestre):
    global model_text
    if not model_text: return "Erro: Modelo de Texto Gemini não inicializado. Verifique a API Key e o nome do modelo."
    try:
        response = model_text.generate_content(prompt_para_o_mestre)
        text_resp = response.text
        return text_resp
    except Exception as e:
        return f"Erro na API Gemini (Texto): {e}"

def parse_initial_setup_from_gemini(response_text):
    data = {"current_objective": "Explorar o desconhecido.", "current_location": "Um lugar misterioso.", "player_inventory": ["Nada de especial"]}
    try:
        if not response_text or not response_text.strip(): return data
        lines = response_text.strip().split('\n')
        for line in lines:
            if line.upper().startswith("OBJETIVO:"): data["current_objective"] = line.split(":", 1)[1].strip()
            elif line.upper().startswith("LOCALIZAÇÃO:") or line.upper().startswith("LOCALIZACAO:"): data["current_location"] = line.split(":", 1)[1].strip()
            elif line.upper().startswith("INVENTÁRIO:") or line.upper().startswith("INVENTARIO:"):
                items_str = line.split(":", 1)[1].strip().replace('[','').replace(']','')
                parsed_items = [item.strip().strip("'\"") for item in items_str.split(',') if item.strip()]
                if parsed_items: data["player_inventory"] = parsed_items
    except Exception as e: print(f"Erro no parse_initial_setup: {e}")
    return data

async def iniciar_aventura_gradio(player_name_input, player_class_choice, custom_class_input, duration_choice_key, gs_dict, ch_list):
    gs = {}
    ch = []
    initial_chatbot_history = []
    log_setup_feedback = []

    gs["player_name"] = player_name_input.strip() or "Aventureiro Anônimo"
    log_setup_feedback.append(f"Nome: {gs['player_name']}.")
    player_class_options = ["Guerreiro Astuto", "Maga Perspicaz", "Ladina Silenciosa", "Explorador Corajoso", "Bardo Eloquente"]
    gs["player_class"] = custom_class_input.strip() or player_class_choice or player_class_options[0]
    log_setup_feedback.append(f"Classe: {gs['player_class']}.")
    opt = DURATION_OPTIONS_CONFIG.get(duration_choice_key, DURATION_OPTIONS_CONFIG["1"])
    gs.update({"chosen_duration_id": opt['id'], "chosen_duration_name": opt['name'], "max_turns": opt['turns'], "current_turn": 0})
    log_setup_feedback.append(f"Duração: {gs['chosen_duration_name']}.")

    prompt_setup = f"""Para RPG: jogador '{gs['player_name']}' ({gs['player_class']}), aventura '{gs['chosen_duration_name']}'. Gere:
OBJETIVO: objetivo inicial.
LOCALIZAÇÃO: local inicial.
INVENTÁRIO: 2-3 itens (item1, item2).
Responda com prefixos OBJETIVO:, LOCALIZAÇÃO:, INVENTÁRIO:."""
    resposta_setup = mestre_responde(prompt_setup)
    if "Erro: Modelo de Texto Gemini não inicializado" in resposta_setup:
        log_setup_feedback.append(resposta_setup)
        initial_chatbot_history.append({"role": "assistant", "content": resposta_setup})
        return gs, ch, initial_chatbot_history, "\n".join(log_setup_feedback), None, gr.update(visible=True), gr.update(visible=False)

    parsed_data = parse_initial_setup_from_gemini(resposta_setup)
    gs.update(parsed_data)
    log_setup_feedback.extend([f"Obj: {gs['current_objective']}", f"Local: {gs['current_location']}", f"Inv: {', '.join(gs['player_inventory'])}"])
    prompt_contexto = f"""GM de RPG. Jogador: '{gs['player_name']}' ({gs['player_class']}). Aventura '{gs['chosen_duration_name']}' (~{gs['max_turns']}t).
Base: Obj: {gs['current_objective']}. Local: {gs['current_location']}. Inv: {', '.join(gs['player_inventory'])}.
Instruções: Narre o cenário (2-3p), integre objetivo, apresente desafio. Convide à ação. APENAS narração inicial.
Comece aventura para {gs['player_name']}:"""
    desc_inicial = mestre_responde(prompt_contexto)
    audio_path = None

    if "Erro:" in desc_inicial or not desc_inicial.strip():
        final_desc = f"Mestre falhou ao iniciar: {desc_inicial or 'Resposta vazia.'}"
        initial_chatbot_history.append({"role": "assistant", "content": final_desc})
        audio_path = await speak_text_for_gradio(final_desc)
        gs["player_name"] = None
        return gs, ch, initial_chatbot_history, "\n".join(log_setup_feedback), audio_path, gr.update(visible=True), gr.update(visible=False)

    initial_chatbot_history.append({"role": "assistant", "content": desc_inicial})
    audio_path = await speak_text_for_gradio(desc_inicial)
    gs["current_situation"] = desc_inicial
    ch.append({'role': 'system', 'parts': [f"Setup: Obj='{gs['current_objective']}', Local='{gs['current_location']}', Inv='{', '.join(gs['player_inventory'])}'."]})
    ch.append({'role': 'model', 'parts': [desc_inicial]})
    return gs, ch, initial_chatbot_history, "Aventura iniciada!", audio_path, gr.update(visible=False), gr.update(visible=True)

async def rodada_do_jogo_gradio(acao_jogador_input, gs_dict, ch_list, current_chatbot_hist_msgs, game_ended_flag_val):
    gs = gs_dict.copy()
    ch = list(ch_list)
    updated_chatbot_hist = list(current_chatbot_hist_msgs)

    if not acao_jogador_input.strip():
        yield gs, ch, updated_chatbot_hist, None, gr.update(interactive=True), "", game_ended_flag_val
        return

    updated_chatbot_hist.append({"role": "user", "content": acao_jogador_input})
    updated_chatbot_hist.append({"role": "assistant", "content": "Mestre (Gemini): Analisando..."})
    yield gs, ch, updated_chatbot_hist, None, gr.update(interactive=False), "", game_ended_flag_val

    gs["current_turn"] = gs.get("current_turn", 0) + 1
    audio_mestre_path = None
    jogo_terminou_nesta_rodada = False

    if acao_jogador_input.lower() == 'sair':
        msg_sair = f"Jornada encerrada por {gs['player_name']}."
        updated_chatbot_hist[-1]["content"] = msg_sair
        audio_mestre_path = await speak_text_for_gradio(msg_sair)
        jogo_terminou_nesta_rodada = True
        yield gs, ch, updated_chatbot_hist, audio_mestre_path, gr.update(interactive=False), "", jogo_terminou_nesta_rodada
        return

    gs["previous_player_actions"] = (gs.get("previous_player_actions", []) + [acao_jogador_input])[-3:]
    instrucao_duracao = ""
    ct, mt, cdn = gs.get('current_turn',0), gs.get('max_turns',1), gs.get('chosen_duration_name','')
    if ct == mt: instrucao_duracao = f"ÚLTIMO TURNO ({ct}/{mt}) de '{cdn}'! CONCLUA o arco."
    elif ct > mt * 0.75: instrucao_duracao = f"Próximo do fim de '{cdn}' (Turno {ct}/{mt}). Guie para clímax."

    hist_fmt_parts = []
    for entry in ch[-4:]:
        role_label = "Jogador" if entry['role'] == 'user' else ("System" if entry['role'] == 'system' else "Mestre")
        content = entry['parts'][0] if isinstance(entry['parts'], list) else entry['parts']
        if entry['role'] != 'system' : hist_fmt_parts.append(f"{role_label}: {content}")
    hist_fmt = "\n".join(hist_fmt_parts)
    prompt_mestre = f"""GM de RPG. Jogador: {gs.get('player_name')} ({gs.get('player_class')}). Aventura: {cdn} ({mt}t), Turno {ct}. {instrucao_duracao}
Histórico Recente:
{hist_fmt or 'Nenhum.'}
Situação Atual: {gs.get('current_situation')}
Ação do Jogador: '{acao_jogador_input}'
Inventário: {', '.join(gs.get('player_inventory',[]))}
Objetivo: {gs.get('current_objective')}
Instruções: Reaja, avance a narrativa, considere duração. Termine com deixa (a menos que final)."""

    resp_mestre = mestre_responde(prompt_mestre)
    if "Erro: Modelo de Texto Gemini não inicializado" in resp_mestre:
        updated_chatbot_hist[-1]["content"] = resp_mestre
        yield gs, ch, updated_chatbot_hist, await speak_text_for_gradio(resp_mestre), gr.update(interactive=True), "", True
        return

    if "Erro:" in resp_mestre:
        resp_mestre_final = f"Mestre com problemas: {resp_mestre}"
        audio_mestre_path = await speak_text_for_gradio(resp_mestre_final)
        gs["current_turn"] -= 1
    else:
        resp_mestre_final = resp_mestre
        audio_mestre_path = await speak_text_for_gradio(resp_mestre_final)
        gs["current_situation"] = resp_mestre_final
        ch.append({'role': 'user', 'parts': [acao_jogador_input]})
        ch.append({'role': 'model', 'parts': [resp_mestre_final]})

    updated_chatbot_hist[-1]["content"] = resp_mestre_final

    if gs.get("current_turn", 0) >= gs.get("max_turns", 1) and not jogo_terminou_nesta_rodada:
        final_msg = f"\nA aventura '{cdn}' chegou ao seu fim no turno {gs['current_turn']}/{gs['max_turns']}."
        updated_chatbot_hist.append({"role": "assistant", "content": final_msg})
        if not audio_mestre_path : audio_mestre_path = await speak_text_for_gradio(final_msg)
        jogo_terminou_nesta_rodada = True

    yield gs, ch, updated_chatbot_hist, audio_mestre_path, gr.update(interactive=not jogo_terminou_nesta_rodada), "", jogo_terminou_nesta_rodada

# --- Interface Gradio ---
with gr.Blocks(theme=gr.themes.Default(), title="RPG com Gemini Aprimorado") as demo:
    gr.HTML(background_music_html)
    game_state_store = gr.State({})
    conversation_history_store = gr.State([])
    game_ended_flag = gr.State(False)
    gr.Markdown("# RPG Textual Aprimorado com IA Gemini e Voz")
    with gr.Tabs():
        with gr.TabItem("Configuração e Jogo"):
            with gr.Row():
                with gr.Column(scale=1):
                    with gr.Column() as api_key_section:
                        gr.Markdown("## 1. Configuração API Gemini")
                        api_key_input_gradio = gr.Textbox(label="Sua Chave API do Gemini", placeholder="Cole sua chave AIza...", type="password", lines=1)
                        submit_api_key_button = gr.Button("Configurar API e Modelo")
                        api_key_feedback = gr.Textbox(label="Feedback da Configuração API", lines=3, interactive=False, max_lines=5)
                    with gr.Column(visible=False) as game_setup_block:
                        gr.Markdown("## 2. Detalhes da Aventura")
                        player_name_gradio = gr.Textbox(label="Nome do Aventureiro(a)", placeholder="Ex: Elara")
                        player_class_options_list = ["Guerreiro Astuto", "Maga Perspicaz", "Ladina Silenciosa", "Explorador Corajoso", "Bardo Eloquente"]
                        player_class_choice_gradio = gr.Radio(player_class_options_list, label="Escolha sua Classe", value=player_class_options_list[0])
                        custom_class_gradio = gr.Textbox(label="Ou crie sua classe", placeholder="Ex: Batedor das Sombras")
                        duration_options_display = [(v['name'], k) for k, v in DURATION_OPTIONS_CONFIG.items()]
                        duration_choice_gradio = gr.Radio(duration_options_display, label="Duração da Aventura", value="1")
                        start_adventure_button = gr.Button("Iniciar Aventura!")
                        game_setup_feedback = gr.Textbox(label="Log de Início", lines=3, interactive=False, max_lines=10)
                        audio_output_setup = gr.Audio(label="Narração Inicial", autoplay=True)
                with gr.Column(scale=2, visible=False) as main_game_block:
                    gr.Markdown("## 3. Sua Aventura")
                    chatbot_component = gr.Chatbot(
                        label="Narrativa da Aventura", height=550, type='messages',
                        avatar_images=(None, "https://www.gstatic.com/lamda/images/gemini_sparkle_v002_darkμορφή.svg"))
                    player_action_input = gr.Textbox(label="O que você faz?", placeholder="Digite sua ação ou 'sair' para terminar...", show_label=False, lines=1)
                    submit_action_button = gr.Button("Enviar Ação")
                    audio_output_game = gr.Audio(label="Narração do Mestre", autoplay=True)

    demo.load(
        load_api_key_on_startup_gradio, inputs=[],
        outputs=[api_key_feedback, api_key_section, game_setup_block, api_key_input_gradio])
    submit_api_key_button.click(
        configure_gemini_api_gradio_direct, inputs=[api_key_input_gradio],
        outputs=[api_key_feedback, api_key_section, game_setup_block])

    def reset_game_state_for_new_adventure():
        return {}, [], False, [], ""

    start_adventure_button.click(
        reset_game_state_for_new_adventure, outputs=[game_state_store, conversation_history_store, game_ended_flag, chatbot_component, player_action_input]
    ).then(
        iniciar_aventura_gradio,
        inputs=[player_name_gradio, player_class_choice_gradio, custom_class_gradio, duration_choice_gradio, game_state_store, conversation_history_store],
        outputs=[game_state_store, conversation_history_store, chatbot_component, game_setup_feedback, audio_output_setup, game_setup_block, main_game_block])

    player_action_outputs = [
        game_state_store, conversation_history_store, chatbot_component, audio_output_game,
        player_action_input, player_action_input, game_ended_flag]

    def handle_game_flow_after_turn(is_game_over_flag):
        global model_text
        if is_game_over_flag:
            api_text_model_configured = bool(model_text)
            return gr.update(visible=not api_text_model_configured), gr.update(visible=api_text_model_configured), gr.update(visible=False)
        return gr.update(), gr.update(), gr.update()

    event_args = {
        "inputs": [player_action_input, game_state_store, conversation_history_store, chatbot_component, game_ended_flag],
        "outputs": player_action_outputs,
        "show_progress": "full"
    }
    submit_action_event = submit_action_button.click(**event_args)
    submit_action_event.then(handle_game_flow_after_turn, inputs=[game_ended_flag], outputs=[api_key_section, game_setup_block, main_game_block])
    input_submit_event = player_action_input.submit(**event_args)
    input_submit_event.then(handle_game_flow_after_turn, inputs=[game_ended_flag], outputs=[api_key_section, game_setup_block, main_game_block])

demo.queue()
demo.launch(debug=True, share=True)